<h3>cobrapy</h3>
cobrapy is a Python binding of the COBRA Toolbox (https://opencobra.github.io/cobratoolbox/latest/), a toolbox for constraint-based metabolic modelling. One of the key features of COBRA is its ability to load SBMLs and to run Flux Balanace Analyses on them.<br>
cobrapy's home page is https://opencobra.github.io/cobrapy/<br>
cobrapy's documentation can be found under https://cobrapy.readthedocs.io/en/stable/<br>
Most of this notebook's examples are based on cobrapy's documentation.<br>
If you use pip, you can install cobrapy by running "pip install cobrapy" and cobrapy's SBML functionalites with "pip install python-libsbml".

<h3>1. Loading a test model</h3>
https://cobrapy.readthedocs.io/en/stable/getting_started.html

In [2]:
import cobra
import cobra.test

model = cobra.test.create_test_model("textbook")

In [9]:
# Note that cobrapy is partially integrated into Jupyter
model

Name,e_coli_core
Memory address,0x022c27685710
Number of metabolites,72
Number of reactions,95
Objective expression,-1.0*Biomass_Ecoli_core_reverse_2cdba + 1.0*Biomass_Ecoli_core
Compartments,"cytosol, extracellular"


In [3]:
print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))

95
72
137


In [5]:
model.reactions[0]

Reaction identifier,ACALD
Name,acetaldehyde dehydrogenase (acetylating)
Memory address,0x022c303dd5c0
Stoichiometry,acald_c + coa_c + nad_c <=> accoa_c + h_c + nadh_c Acetaldehyde + Coenzyme A + Nicotinamide adenine dinucleotide <=> Acetyl-CoA + H+ + Nicotinamide adenine dinucleotide - reduced
GPR,b0351 or b1241
Lower bound,-1000.0
Upper bound,1000.0


In [12]:
model.reactions[0].reversibility

True

In [14]:
pgi = model.reactions.get_by_id("PGI")
pgi.add_metabolites({model.metabolites.get_by_id("h_c"): -1})
pgi.check_mass_balance()
pgi.subtract_metabolites({model.metabolites.get_by_id("h_c"): -1})

In [6]:
model.metabolites[-1]

Metabolite identifier,xu5p__D_c
Name,D-Xylulose 5-phosphate
Memory address,0x022c303b6c88
Formula,C5H9O8P
Compartment,c
In 3 reaction(s),"RPE, TKT1, TKT2"


In [10]:
model.genes.get_by_id("b0351")

Gene identifier,b0351
Name,mhpF
Memory address,0x022c303bb128
Functional,True
In 1 reaction(s),ACALD


In [17]:
# Efficient knock-outs on-the-fly, without copying the model
for reaction in model.reactions[:5]:
    with model as model:
        reaction.knock_out()
        model.optimize()
        print('%s blocked (bounds: %s), new growth rate %f' %
              (reaction.id, str(reaction.bounds), model.objective.value))

ACALD blocked (bounds: (0, 0)), new growth rate 0.873922
ACALDt blocked (bounds: (0, 0)), new growth rate 0.873922
ACKr blocked (bounds: (0, 0)), new growth rate 0.873922
ACONTa blocked (bounds: (0, 0)), new growth rate -0.000000
ACONTb blocked (bounds: (0, 0)), new growth rate -0.000000


<h3>2. Write and Load an SBML</h3>
https://cobrapy.readthedocs.io/en/stable/io.html

In [20]:
# Write the model of section 1 into an SBML
cobra.io.write_sbml_model(model, "./cobrapy/textbook.xml")

In [24]:
# Read the model great again
model = cobra.io.read_sbml_model("./cobrapy/textbook.xml")
model

Name,e_coli_core
Memory address,0x022c29d72550
Number of metabolites,72
Number of reactions,95
Objective expression,-1.0*Biomass_Ecoli_core_reverse_2cdba + 1.0*Biomass_Ecoli_core
Compartments,"cytosol, extracellular"


<h3>3. Performing FBA</h3>
https://cobrapy.readthedocs.io/en/stable/simulating.html

In [28]:
fba_solution = model.optimize()
# model.slim_optimize() if we are only interested in the objective value
print(fba_solution)

<Solution 0.874 at 0x22c315999e8>


In [27]:
fba_solution.objective_value

0.8739215069684307

<h3>4. Analyzing FBA results</h3>

In [29]:
model.summary()

IN FLUXES        OUT FLUXES    OBJECTIVES
---------------  ------------  ----------------------
o2_e      21.8   h2o_e  29.2   Biomass_Ecol...  0.874
glc__D_e  10     co2_e  22.8
nh4_e      4.77  h_e    17.5
pi_e       3.21


In [30]:
# Redox balance
model.metabolites.nadh_c.summary()

PRODUCING REACTIONS -- Nicotinamide adenine dinucleotide - reduced (nadh_c)
---------------------------------------------------------------------------
%       FLUX  RXN ID      REACTION
----  ------  ----------  --------------------------------------------------
42%    16     GAPD        g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c
24%     9.28  PDH         coa_c + nad_c + pyr_c --> accoa_c + co2_c + nadh_c
13%     5.06  AKGDH       akg_c + coa_c + nad_c --> co2_c + nadh_c + succ...
13%     5.06  MDH         mal__L_c + nad_c <=> h_c + nadh_c + oaa_c
8%      3.1   Biomass...  1.496 3pg_c + 3.7478 accoa_c + 59.81 atp_c + 0....

CONSUMING REACTIONS -- Nicotinamide adenine dinucleotide - reduced (nadh_c)
---------------------------------------------------------------------------
%       FLUX  RXN ID      REACTION
----  ------  ----------  --------------------------------------------------
100%   38.5   NADH16      4.0 h_c + nadh_c + q8_c --> 3.0 h_e + nad_c + q...


<h3>5. Sections 3 and 4 together with a new objective function</h3>
https://cobrapy.readthedocs.io/en/stable/simulating.html#Changing-the-Objectives

In [32]:
# Currently, a biomass function is the objective function...
from cobra.util.solver import linear_reaction_coefficients
linear_reaction_coefficients(model)

{<Reaction Biomass_Ecoli_core at 0x22c317032e8>: 1.0}

In [34]:
# ...which is set as a reaction
model.reactions.get_by_id("Biomass_Ecoli_core")

Reaction identifier,Biomass_Ecoli_core
Name,Biomass Objective Function with GAM
Memory address,0x022c317032e8
Stoichiometry,1.496 3pg_c + 3.7478 accoa_c + 59.81 atp_c + 0.361 e4p_c + 0.0709 f6p_c + 0.129 g3p_c + 0.205 g6p_c + 0.2557 gln__L_c + 4.9414 glu__L_c + 59.81 h2o_c + 3.547 nad_c + 13.0279 nadph_c + 1.7867 oaa_c + 0.5191 pep_c + 2.8328 pyr_c + 0.8977 r5p_c --> 59.81 adp_c + 4.1182 akg_c + 3.7478 coa_c + 59.81 h_c + 3.547 nadh_c + 13.0279 nadp_c + 59.81 pi_c 1.496 3-Phospho-D-glycerate + 3.7478 Acetyl-CoA + 59.81 ATP + 0.361 D-Erythrose 4-phosphate + 0.0709 D-Fructose 6-phosphate + 0.129 Glyceraldehyde 3-phosphate + 0.205 D-Glucose 6-phosphate + 0.2557 L-Glutamine + 4.9414 L-Glutamate + 59.81 H2O + 3.547 Nicotinamide adenine dinucleotide + 13.0279 Nicotinamide adenine dinucleotide phosphate - reduced + 1.7867 Oxaloacetate + 0.5191 Phosphoenolpyruvate + 2.8328 Pyruvate + 0.8977 alpha-D-Ribose 5-phosphate --> 59.81 ADP + 4.1182 2-Oxoglutarate + 3.7478 Coenzyme A + 59.81 H+ + 3.547 Nicotinamide adenine dinucleotide - reduced + 13.0279 Nicotinamide adenine dinucleotide phosphate + 59.81 Phosphate
GPR,
Lower bound,0.0
Upper bound,1000.0


In [35]:
# Now, we set the objective to ATPM
model.objective = "ATPM"
model.reactions.get_by_id("ATPM")

Reaction identifier,ATPM
Name,ATP maintenance requirement
Memory address,0x022c317032b0
Stoichiometry,atp_c + h2o_c --> adp_c + h_c + pi_c ATP + H2O --> ADP + H+ + Phosphate
GPR,
Lower bound,8.39
Upper bound,1000.0


In [36]:
linear_reaction_coefficients(model)

{<Reaction ATPM at 0x22c317032b0>: 1.0}

In [40]:
# Perform FBA with the new objective function
model.optimize()
model.summary()

IN FLUXES     OUT FLUXES    OBJECTIVES
------------  ------------  ------------
o2_e      60  co2_e  60     ATPM  175
glc__D_e  10  h2o_e  60


<h3>6. Loopless FBA</h3>
Some solutions may contain loops, which should not be possible in vivo. This can be solved by using solvers which use looplessness as a contstrain. However, loopless solutions are generally more computationally expensive.<br>
https://cobrapy.readthedocs.io/en/stable/loopless.html<br>
Paper: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3030201/

In [59]:
from cobra.flux_analysis.loopless import add_loopless, loopless_solution

loopless = loopless_solution(model)
loopless[:5]

ACALD      0.0
ACALDt     0.0
ACKr       0.0
ACONTa    20.0
ACONTb    20.0
Name: fluxes, dtype: float64

<h3>7. Flux Variability Analysis</h3>
As an FBA does not always return a single most optimal solution, FBA returns the bounds of all optimal solutions.
https://cobrapy.readthedocs.io/en/stable/simulating.html#Running-FVA

In [42]:
from cobra.flux_analysis import flux_variability_analysis

In [45]:
flux_variability_analysis(model, model.reactions[:5])

,maximum,minimum
ACALD,0.0,-2.623542e-14
ACALDt,0.0,-2.623542e-14
ACKr,0.0,-4.012477e-14
ACONTa,20.0,2.000000e+01
ACONTb,20.0,2.000000e+01


In [49]:
# With 90 % optimality
flux_variability_analysis(model, model.reactions[:5], fraction_of_optimum=0.9)

,maximum,minimum
ACALD,0.000000e+00,-2.692308
ACALDt,0.000000e+00,-2.692308
ACKr,9.876464e-16,-4.117647
ACONTa,2.000000e+01,8.461538
ACONTb,2.000000e+01,8.461538


In [48]:
model.summary(fva=.95)

IN FLUXES                     OUT FLUXES                    OBJECTIVES
----------------------------  ----------------------------  ------------
id          Flux  Range       id          Flux  Range       ATPM  175
--------  ------  ----------  --------  ------  ----------
o2_e          60  [55.9, 60]  co2_e         60  [54.2, 60]
glc__D_e      10  [9.5, 10]   h2o_e         60  [54.2, 60]
nh4_e          0  [0, 0.673]  for_e          0  [0, 5.83]
pi_e           0  [0, 0.171]  h_e            0  [0, 5.83]
                              ac_e           0  [0, 2.06]
                              acald_e        0  [0, 1.35]
                              pyr_e          0  [0, 1.35]
                              etoh_e         0  [0, 1.17]
                              lac__D_e       0  [0, 1.13]
                              succ_e         0  [0, 0.875]
                              akg_e          0  [0, 0.745]
                              glu__L_e       0  [0, 0.673]


In [55]:
# Loops can be avoided by setting loopless=True
loop_reactions = [model.reactions.FRD7, model.reactions.SUCDi]
flux_variability_analysis(model, loop_reactions, loopless=False)

,maximum,minimum
FRD7,980.0,0.0
SUCDi,1000.0,20.0


In [56]:
flux_variability_analysis(model, loop_reactions, loopless=True)

,maximum,minimum
FRD7,0.0,0.0
SUCDi,20.0,20.0


<h3>8. Parsimoniuous FBA</h3>
This is an FBA with the additional contrains of miminal total fluxes.<br>
https://cobrapy.readthedocs.io/en/stable/simulating.html#Running-pFBA

In [67]:
cobra.flux_analysis.pfba(model)[:5]

ACALD     0.000000e+00
ACALDt    0.000000e+00
ACKr     -4.012477e-14
ACONTa    2.000000e+01
ACONTb    2.000000e+01
Name: fluxes, dtype: float64

<h3>9. More analyses</h3>
Once you have a good model, you can use it for more analyses:<br>
Production envelopes (i.e. growth in dependence of the nutrients): https://cobrapy.readthedocs.io/en/stable/phenotype_phase_plane.html<br>
Gapfilling (i.e. finding necessary reactions for an objective function): https://cobrapy.readthedocs.io/en/stable/gapfilling.html

PSB 2017